In [79]:
import os
os.chdir(r'F:\machine learning\lab 2\timeseires')


In [131]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [133]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [35]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [37]:
model1 = create_rnn()
model1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 24, 8)               │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ (None, 20)                  │             580 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
tensorflow.keras.utils.plot_model(model1 )

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [41]:
checkpoints = "F:\\machine learning\\lab 2\\lab 7\\check_points\\best_model.keras"
OUTPUT_PATH = r'F:\machine learning\lab 2\lab 7\check_points'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Admin\AppData\Local\Temp\ipykernel_8000\876469152.py:3: SyntaxWarning: invalid escape sequence '\h'
  FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
C:\Users\Admin\AppData\Local\Temp\ipykernel_8000\876469152.py:4: SyntaxWarning: invalid escape sequence '\h'
  JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])


In [83]:
from keras.callbacks import ModelCheckpoint

# Corrected checkpoint path
checkpoints = "F:\\machine learning\\lab 2\\lab 7\\check_points\\best_model.keras"

# Construct the callback to save only the *best* model based on validation loss
EpochCheckpoint1 = ModelCheckpoint(
    checkpoints,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)



# Construct the set of callbacks
callbacks = [EpochCheckpoint1,]


In [85]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [93]:
import os
path_dataset ='F::\machine learning\lab 2\lab 7\1_Orignal_AEP_hourly.csv'
path_tr = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\m'
<>:6: SyntaxWarning: invalid escape sequence '\m'
<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:13: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\m'
<>:6: SyntaxWarning: invalid escape sequence '\m'
<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:13: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Admin\AppData\Local\Temp\ipykernel_8000\2404538863.py:2: SyntaxWarning: invalid escape sequence '\m'
  path_dataset ='F::\machine learning\lab 2\lab 7\1_Orignal_AEP_hourly.csv'
C:\Users\Admin\AppData\Local\Temp\ipykernel_8000\2404538863.py:3: SyntaxWarning: invalid escape sequence '\m'
  path_tr = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_train.csv')
C:\Users\Admin\AppData\Local\Temp\ipykernel_8000\2404538863.py:6: SyntaxWarning: invalid escape sequence '\m'
  pa

((84907, 21), (24259, 21), (12130, 21))

In [95]:
time_steps=24
num_features=21

In [97]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.6553351879119873 sec


In [99]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/60
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0760 - mae: 0.0760 - mape: 5480.9458
Epoch 1: val_loss improved from inf to 0.01326, saving model to F:\machine learning\lab 2\lab 7\check_points\best_model.keras
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 29s 10ms/step - loss: 0.0759 - mae: 0.0759 - mape: 5473.1919 - val_loss: 0.0133 - val_mae: 0.0133 - val_mape: 6.5444
Epoch 2/60
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0139 - mae: 0.0139 - mape: 265.3095
Epoch 2: val_loss improved from 0.01326 to 0.01243, saving model to F:\machine learning\lab 2\lab 7\check_points\best_model.keras
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - loss: 0.0139 - mae: 0.0139 - mape: 265.2024 - val_loss: 0.0124 - val_mae: 0.0124 - val_mape: 6.3033
Epoch 3/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0120 - mae: 0.0120 - mape: 33.8489
Epoch 3: val_loss improved from 0.01243 to 0.01111, saving model to F:\machine learning\lab 2\lab 7\check_points\best_model.keras
2653/2653 ━━━━━━━━━

In [101]:

model = load_model(r'F:\machine learning\lab 2\lab 7\check_points\best_model.keras')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Mean Absolute Error (MAE): 107.23
Median Absolute Error (MedAE): 84.75
Mean Squared Error (MSE): 20298.21
Root Mean Squared Error (RMSE): 142.47
Mean Absolute Percentage Error (MAPE): 0.73 %
Median Absolute Percentage Error (MDAPE): 0.58 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [113]:
checkpoints = r'F:\machine learning\lab 2\lab 7\check_points.keras'
model=r'‪F:\machine learning\lab 2\lab 7'
start_epoch= 54

In [125]:
from tensorflow.keras.models import load_model
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

model_path = r"F:\machine learning\lab 2\lab 7\check_points\best_model.keras"

if not os.path.exists(model_path):
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])
else:
    print("[INFO] loading model from {}...".format(model_path))
    model = load_model(model_path)

    # Fix learning rate handling
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))
    if not isinstance(model.optimizer.learning_rate, tf.Variable):
        model.optimizer.learning_rate = tf.Variable(model.optimizer.learning_rate)
    model.optimizer.learning_rate.assign(1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))


[INFO] loading model from F:\machine learning\lab 2\lab 7\check_points\best_model.keras...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [127]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0064 - mae: 0.0064 - mape: 27.6022
Epoch 1: val_loss improved from inf to 0.00652, saving model to F:\machine learning\lab 2\lab 7\check_points\best_model.keras
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - loss: 0.0064 - mae: 0.0064 - mape: 27.9520 - val_loss: 0.0065 - val_mae: 0.0065 - val_mape: 3.0364
Epoch 2/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0063 - mae: 0.0063 - mape: 24.5504
Epoch 2: val_loss improved from 0.00652 to 0.00646, saving model to F:\machine learning\lab 2\lab 7\check_points\best_model.keras
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 36s 13ms/step - loss: 0.0063 - mae: 0.0063 - mape: 24.6406 - val_loss: 0.0065 - val_mae: 0.0065 - val_mape: 2.8489
Epoch 3/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0063 - mae: 0.0063 - mape: 426.6573
Epoch 3: val_loss did not improve from 0.00646
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - loss: 0.0063 - mae: 0.0063 - mape: 426.5455 - val_loss: 0

In [129]:

model = load_model(r'F:\machine learning\lab 2\lab 7\check_points\best_model.keras')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
Mean Absolute Error (MAE): 103.05
Median Absolute Error (MedAE): 80.9
Mean Squared Error (MSE): 18862.03
Root Mean Squared Error (RMSE): 137.34
Mean Absolute Percentage Error (MAPE): 0.71 %
Median Absolute Percentage Error (MDAPE): 0.56 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
